# Klassifikation

Du hast im letzten Teil schon gesehen, wie sich Daten von Reddit herunterladen lassen. Auch die Flairs hast du kennengelernt.

Allerdings hast du auch gesehen, dass nicht für alle Posts die Flairs richtig bereitgestellt werden:
* Es gibt alte Posts, bei denen die Flairs noch nicht eingeführt waren
* Nicht alle Autoren verwenden die Flairs konsistent

Mithilfe der Klassifikation hast du aber die Möglichkeite, auch die bisher nicht kategorisierten Daten noch den Flairs richtig zuzuordnen. Das wirst du in diesem Notebook durchführen.

## Achtung

Dieser Teil des Notebooks benötigt das gesamte Technology-Subreddit, das für den Download leider zu groß ist (> 10 GB). Daher ist dies im Colab-Notebook nicht ablauffähig! Eine Erklärung, wie du diese Daten selbst akquirieren kannst, findest du z.B. im iX-Artikel [Beziehungssache](https://www.heise.de/select/ix/2021/7/2102513144636338770). 

Du benötigst dies aber nur für die Erzeugung einer kleineren Datenmenge, die du dann anschließend im Notebook ausschließlich verwenden wirst und selbstverständlich dazu herunterladen kannst.

Bitte ab hier deshalb vorerst nur zuschauen!

In [1]:
import pandas as pd

Zunächst lädst du alle Posts ein. Viele Felder benötigst du nicht, um Speicher zu sparen, benutzt du nur die notwendigen.

In [2]:
posts = pd.read_csv("technology.csv.xz", parse_dates=["created_utc"])

/tmp/ipykernel_79859/33992809.py:1: DtypeWarning: Columns (16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  posts = pd.read_csv("technology.csv.xz", parse_dates=["created_utc"])


Flairs wurden erst 2015 eingeführt und gelten nur für Toplevel-Posts. Du möchtest nur Posts verwenden, bei denen ein Flair gesetzt wurde, sonst könnten die Autoren das evtl. vergessen haben. Da du den `DataFrame` später noch modifizieren wirst, erzeugst du eine Kopie.

In [3]:
top2015 = posts[(posts["created_utc"].dt.year>=2015) & 
                posts["parent_id"].isna() & 
                ~posts["flair"].isna()].copy()

Du bestimmst nun, ob die Posts zu Transportation gehören oder nicht. Beachte dabei, dass die Flairs *Transport* und *Transportation* genannt wurden.

In [4]:
top2015["target"] = top2015["flair"].isin(["Transport", "Transportation"])

Wie verteilen sich die positiven und negativen Beispiele?

In [5]:
top2015.value_counts("target")

target
False    448991
True      14888
Name: count, dtype: int64

Daraus erzeugst du ein stratifiziertes Datenset mit gleich vielen positiven wie negativen Beispielen: 

In [6]:
pos = top2015[top2015["target"] == True]
neg = top2015[top2015["target"] == False]
data = pd.concat([pos, neg.sample(n = len(pos), random_state=42)], 
                 ignore_index=True)

Nun werden die Daten vektorisiert. Du verzichtest dabei auf Stopwords etc., die üblicherweise in der Textanalyse verwendet werden und vertraust auf die Funktionsweise von TF/IDF:

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1,1), max_df=0.7, min_df=5)
tfidf_vectors = tfidf.fit_transform(data["title"])

Als Konvention nennst du die unabhängige Variable `X` und die abhängige `y`:

In [8]:
X = tfidf_vectors
y = data["target"].values

### Hold-out-Verfahren: Getrennte Mengen für Training und Test

Du teilst die Datenmenge in einen Teil, mit dem du den Klassifikator trainierst (75%) und eine, mit dem du die Ergebnisse verifizierst:

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Das Klassifikationsmodell wird als Support Vector Machine nur mit Trainingsdaten trainiert:

In [10]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
clf.fit(X_train, y_train)

SGDClassifier(random_state=42)

Du führst eine Vorhersage für die (dem Klassifikator unbekannten) Testdaten durch:

In [11]:
y_predicted = clf.predict(X_test)

Und betrachtest die Ergebnisse der Klassifikation:

In [12]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

       False       0.90      0.96      0.93      3696
        True       0.95      0.90      0.93      3748

    accuracy                           0.93      7444
   macro avg       0.93      0.93      0.93      7444
weighted avg       0.93      0.93      0.93      7444



90% bzw. 96% Precision und Recall sind ziemlich gut, mit diesem Modell kannst du arbeiten und nun alle Posts (größtenteils richtig) klassifizieren:

## Gesamtes Datenset klassifizieren

Nun möchtest du die Trainingsdaten nutzen, um das gesamte oben eingelesene Datenset zu klassifizieren. Auch in den Kommentaren können nützliche Informationen enthalten sein, deswegen nutzt du die auch.

Dazu musst du zunächst den Text für alle Posts bestimmen. Dazu fügst du die einzelnen Textkomponenten mit Leerzeichen zusammen:

In [13]:
posts["text"] = posts["title"].map(str) + " " + posts["body"].map(str) + " " + posts["selftext"].map(str)

Nun wendest du den Klassifikator an und klassifizierst alle Posts:

In [14]:
posts["transport"] = clf.predict(tfidf.transform(posts["text"].map(str)))

Du interessierst dich für die Posts, die der Klassifikator als zu "Transport" zugehörig erkennt:

In [15]:
transport = posts[posts["transport"] == True].copy()
transport

,id,kind,title,link_id,parent_id,name,ups,downs,score,author,...,created_utc,permalink,url,selftext,body,flair,level,top_parent,text,transport
231,t1_gkqkyue,t1,NaN,t3_l4cf2u,t1_gkousob,t1_gkqkyue,1,0,1,Marimbalogy,...,2021-01-25 20:00:04,/r/technology/comments/l4cf2u/acting_fcc_chair...,NaN,NaN,It routes traffic through their public hotspot...,NaN,NaN,t3_l4cf2u,nan It routes traffic through their public hot...,True
327,t1_gkpyvd3,t1,NaN,t3_l4cf2u,t1_gkpgh1m,t1_gkpyvd3,1,0,1,m0ondoggy,...,2021-01-25 17:21:48,/r/technology/comments/l4cf2u/acting_fcc_chair...,NaN,NaN,You're not wrong. I don't expect anything to ...,NaN,NaN,t3_l4cf2u,nan You're not wrong. I don't expect anything...,True
391,t1_gkoq0x1,t1,NaN,t3_l4cf2u,t1_gkone5l,t1_gkoq0x1,2,0,2,eddiepaperhands,...,2021-01-25 08:40:14,/r/technology/comments/l4cf2u/acting_fcc_chair...,NaN,NaN,Lol I probably deserve it,NaN,NaN,t3_l4cf2u,nan Lol I probably deserve it nan,True
438,t1_gkpguc7,t1,NaN,t3_l4cf2u,t1_gkorofs,t1_gkpguc7,1,0,1,somebody12344,...,2021-01-25 15:01:19,/r/technology/comments/l4cf2u/acting_fcc_chair...,NaN,NaN,lot's of people watch public tv still along wi...,NaN,NaN,t3_l4cf2u,nan lot's of people watch public tv still alon...,True
683,t1_gkokrzi,t1,NaN,t3_l4cf2u,t1_gknweg7,t1_gkokrzi,1,0,1,TheDude-Esquire,...,2021-01-25 07:24:23,/r/technology/comments/l4cf2u/acting_fcc_chair...,NaN,NaN,The line says could save bet neutrality. Not w...,NaN,NaN,t3_l4cf2u,nan The line says could save bet neutrality. N...,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22330637,t1_h2py9md,t1,NaN,t3_o5iwgi,t1_h2p5oe0,t1_h2py9md,1,0,1,thisLysol,...,2021-06-23 03:45:02,/r/technology/comments/o5iwgi/the_problem_isnt...,NaN,NaN,Public transit here means a 2 hour one way com...,NaN,NaN,NaN,nan Public transit here means a 2 hour one way...,True
22330639,t1_h2rv754,t1,NaN,t3_o5iwgi,t1_h2rechj,t1_h2rv754,1,0,1,thisLysol,...,2021-06-23 16:54:44,/r/technology/comments/o5iwgi/the_problem_isnt...,NaN,NaN,"I specified ""here"". At least in the USA, the v...",NaN,NaN,NaN,"nan I specified ""here"". At least in the USA, t...",True
22330685,t1_h2oljjp,t1,NaN,t3_o5iwgi,t3_o5iwgi,t1_h2oljjp,4,0,4,peanut-butter-kitten,...,2021-06-22 21:02:22,/r/technology/comments/o5iwgi/the_problem_isnt...,NaN,NaN,Commuting is horrible for everyone and the env...,NaN,NaN,NaN,nan Commuting is horrible for everyone and the...,True
22330747,t1_h2pxque,t1,NaN,t3_o5iwgi,t1_h2ppp0u,t1_h2pxque,3,0,3,BadLuckCharm1966,...,2021-06-23 03:40:31,/r/technology/comments/o5iwgi/the_problem_isnt...,NaN,NaN,"True. My husband is one of them. But, when so ...",NaN,NaN,NaN,"nan True. My husband is one of them. But, when...",True


Die Daten speicherst du nun in einer CSV-Datei ab. Vorher ersetzt du noch die Umbrüche durch Leerzeichen, damit das CSV-File auch wieder richtig eingelesen werden kann.

In [16]:
transport["text"] = transport["text"].str.replace("\n", " ")
transport["text"] = transport["text"].str.replace("\r", " ")

In [17]:
transport.set_index("id")[["created_utc", "url", "parent_id", "author", "score", "text"]].\
          to_csv("transport-all-comments.csv.gz", index_label="id")

## Zwischenergebnis

Nach all der Vorarbeit hast du  nun die Daten selektiert, mit denen du jetzt ausschließlich arbeiten wirst. 

Der Vorbereitungsaufwand mag hoch erscheinen, allerdings hast du nun auch wirklich ein Datenset, was genau zu deinen Geschäftsanforderungen passt. Mit manuellen Methoden wäre das nicht mit vertretbarem Aufwand möglich gewesen! 

Das ist auch ein Grund für die Beliebtheit von Data Science: mit relativ moderatem Aufwand können genau passende Datenmenge erzeugt werden!

Die Transport-Daten sind in einer SQLite-Datenbank abgespeichert, mit der du ab jetzt weiterarbeiten wirst. 